In [1]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import t, norm
from joblib import Parallel, delayed

In [2]:
data = pd.read_csv('data.csv')

num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

In [3]:
def process_obs(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    return np.sum(loss), np.var(loss)

In [4]:
shift = 2.0 
factors_tail = np.random.normal(shift, 1, (num_obvs, len(r) + len(data)))
antithetic_factors_tail = -factors_tail
combined_factors_tail = np.vstack([factors_tail, antithetic_factors_tail])


answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors_tail)
answers = np.array(answers)

sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 2192 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 10256 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 20624 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 33296 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 48272 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 65552 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done 85136 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done 107024 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done 131216 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 157712 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 186512 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 200000 out of 200000 | elapsed:  1.4min finished


In [5]:
VaR_antithetic = np.percentile(-np.array(sample_losses), 99.9)

In [6]:
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)

Średnia wariancji strat: 129147.69240639386
VaR antetyczny (100-ty wynik): 0.0
